# Foreground database import

This notebook contains the required steps to import the foreground datatabase.
The database comes originally as an [excel file](Foreground.xlsx).


In [1]:
from pathlib import Path
from pprint import pprint

In [2]:
import bw2data as bd
import bw2io as bi

In [3]:
from project_details import EI_DB_NAME, PROJECT_NAME

In [4]:
bd.projects.set_current(PROJECT_NAME)
bd.databases

Databases dictionary with 2 object(s):
	biosphere3
	ecoinvent-3.9.1-cutoff

In [5]:
# Is the background database name the same as the one we wrote in `propject_details.py`?
assert EI_DB_NAME in bd.databases

## Extract the foreground inventory data

In [6]:
# Include the path to the foreground database
fg_db = Path("./Foreground.xlsx")

# Import your LCI
lci = bi.ExcelImporter(fg_db)

Extracted 1 worksheets in 0.02 seconds


## Transform the data from xlsx format to Brightway format

### Match the foreground database to itself

In [7]:
# Need to match FG_DB to itself
lci.match_database(fields=["name", "unit", "location"])

Applying strategy: link_iterable_by_fields


### Match the FG_DB to the background db

In [8]:
# Need to match FG_DB to the biosphere
lci.match_database(EI_DB_NAME, fields=["name", "reference product", "location", "unit"])

Applying strategy: link_iterable_by_fields


### Match the FG_DB to the biosphere3 database

In [9]:
# Need to match FG_DB to the biosphere
lci.match_database("biosphere3", fields=["name", "categories"])

Applying strategy: link_iterable_by_fields


In [10]:
bi.create_core_migrations()
# Once your package is imported we need to apply strategies
lci.apply_strategies()

# We need to match databases - name and categories but ATTENTION! the categories in
# the excel file is "None" so we willlci.write_excel() also need to match against unit.

lci.statistics()

Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 6.74 seconds
20 datasets
118 exchanges
0 unlinked exchanges
  


(20, 118, 0)

In [11]:
import_statistics = lci.statistics(print_stats=False)

# If all exchanges have been linked, proceed to write the database
if import_statistics[2] == 0:
    lci.write_database()
else:
    print(f"There are unlinked exchanges {import_statistics}")
    lci.write_excel()
    for unlinked_exchange in lci.unlinked:
        pprint(unlinked_exchange)

bd.databases

Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 49666.12it/s]

Vacuuming database 


Created database: asphalt


Databases dictionary with 3 object(s):
	asphalt
	biosphere3
	ecoinvent-3.9.1-cutoff